# Import packages

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
import pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *

import zipfile 
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load dataframe test

In [ ]:
test = pd.read_csv('../input/histopathologic-cancer-detection/sample_submission.csv')

print('Test Set Size:', test.shape)

In [ ]:
test.head()

In [ ]:
#The id in the csv file does not have .tif extension, therefore we need to add it here.
test['id'] = test['id'].apply(lambda x:f'{x}.tif')
test.head()

# Extract images

In [ ]:
test_path = "../input/histopathologic-cancer-detection/test"
print('Test Images:', len(os.listdir(test_path)))

# Data generator

In [ ]:
BATCH_SIZE = 64

test_datagen = ImageDataGenerator(rescale=1/255)

test_loader = test_datagen.flow_from_dataframe(
    dataframe = test,
    directory = test_path,
    x_col = 'id',
    batch_size = BATCH_SIZE,
    shuffle = False,
    class_mode = None,
    target_size = (64,64)
)

# Load model

In [ ]:
cnn_model = keras.models.load_model('../input/lk-cancer-detection-model-v01revised/cancer_model_v08.h5')
cnn_model.summary()

# Test predictions

In [ ]:
test_probs = cnn_model.predict(test_loader)
print(test_probs.shape)

In [ ]:
test_pred = np.argmax(test_probs, axis=1)
print(test_pred[:10]) 

# Prepare submission

In [ ]:
submission = pd.read_csv('../input/histopathologic-cancer-detection/sample_submission.csv')
submission.head()

In [ ]:
submission.label = test_pred
submission.head()

In [ ]:
submission.to_csv('submission.csv', header=True, index=False)